In [11]:
import socket
import time
import ipywidgets as widgets
from IPython.display import display

In [2]:
def test_gqrx_connection(host='localhost', port=7356):
    try:
        # Create a socket and connect to Gqrx
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        s.connect((host, port))

        # Send a command to get the frequency
        s.sendall(b'f\n')
        frequency = s.recv(1024).decode('utf-8').strip()

        s.close()
        return f'Connected successfully! Current frequency: {frequency} Hz'
    except Exception as e:
        return f'Failed to connect: {e}'

In [3]:
    
class GqrxController:
    def __init__(self, host='localhost', port=7356):
        self.host = host
        self.port = port
        self.sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.sock.connect((self.host, self.port))

    def send_command(self, cmd):
        self.sock.sendall((cmd + '\n').encode('utf-8'))
        response = self.sock.recv(1024).decode('utf-8').strip()
        return response

    def get_frequency(self):
        return float(self.send_command('f'))

    def set_frequency(self, freq):
        self.send_command(f'F {freq}')
        return self.get_frequency()

    def close(self):
        self.sock.close()


In [4]:

# Test the connection
test_gqrx_connection()

# Create an instance of GqrxController
gqrx = GqrxController()

# Define a function to update frequency
def update_frequency(freq):
    gqrx.set_frequency(freq)
    print(f"Frequency set to: {freq} Hz")


In [5]:

# Create a slider for frequency control in Wi-Fi channel 11
freq_slider = widgets.FloatSlider(
    value=gqrx.get_frequency(),
    min=2.450e9,  # 2.45 GHz
    max=2.473e9,  # 2.47 GHz
    step=100,  # 1 MHz step
    description='Frequency:',
    continuous_update=False
)
widgets.interactive(update_frequency, freq=freq_slider)


interactive(children=(FloatSlider(value=2450000000.0, continuous_update=False, description='Frequency:', max=2…

In [6]:
import pyaudio
import numpy as np
import matplotlib.pyplot as plt

In [13]:
p = pyaudio.PyAudio()

print("Available audio devices:")
for i in range(p.get_device_count()):
    device_info = p.get_device_info_by_index(i)
    print(f"Index: {i}, Name: {device_info['name']}, Channels: {device_info['maxInputChannels']}, Sample Rate: {device_info['defaultSampleRate']}")


Available audio devices:
Index: 0, Name: l e t h e x (2) Microphone, Channels: 1, Sample Rate: 48000.0
Index: 1, Name: BlackHole 16ch, Channels: 16, Sample Rate: 44100.0
Index: 2, Name: BlackHole 2ch, Channels: 2, Sample Rate: 48000.0
Index: 3, Name: MacBook Air Microphone, Channels: 1, Sample Rate: 44100.0
Index: 4, Name: MacBook Air Speakers, Channels: 0, Sample Rate: 44100.0
Index: 5, Name: MJAudioRecorder, Channels: 2, Sample Rate: 48000.0
Index: 6, Name: Microsoft Teams Audio, Channels: 2, Sample Rate: 48000.0
Index: 7, Name: Serato Virtual Audio, Channels: 2, Sample Rate: 48000.0
Index: 8, Name: rekordbox Aggregate Device, Channels: 0, Sample Rate: 0.0
Index: 9, Name: ZoomAudioDevice, Channels: 2, Sample Rate: 44100.0
Index: 10, Name: SC->Ableton->OBS, Channels: 18, Sample Rate: 48000.0
Index: 11, Name: SDR Monitor, Channels: 2, Sample Rate: 48000.0
Index: 12, Name: Stereo 2.1, Channels: 2, Sample Rate: 48000.0
Index: 13, Name: Multi-Output Device, Channels: 0, Sample Rate: 44100

In [ ]:
def stream_audio(high_pitch_min=1000, high_pitch_max=16000, num_channels=2, num_samples=1024, sample_rate=48000, device_index=0):
    # Set up the audio stream
    p = pyaudio.PyAudio()

    stream = p.open(format=pyaudio.paInt16, channels=num_channels, rate=sample_rate, input=True, frames_per_buffer=num_samples, input_device_index=device_index)

   
    previous_peak_freq = 0 # Initialize a variable to keep track of the previous peak frequency

    try:
        while True:
            try:
                # Read a chunk of audio data
                data = np.frombuffer(stream.read(num_samples, exception_on_overflow=False), dtype=np.int16)
            except OSError as e:
                if e.errno == -9981:
                    print("Input overflowed; skipping this chunk")
                    continue

            # Calculate the amplitude
            amplitude = np.abs(data).mean()

            # Perform a Fourier transform to get the frequency spectrum
            spectrum = np.fft.fft(data)
            freqs = np.fft.fftfreq(len(spectrum), 1/sample_rate) # Include the sampling rate to get frequencies in Hz
            magnitude = np.abs(spectrum)

            # Find the peak frequency
            peak_freq = np.abs(freqs[np.argmax(magnitude[:len(magnitude)//2])]) # Only consider the positive frequencies

            # Check if the frequency shift is higher than 1 Hz
            if abs(peak_freq - previous_peak_freq) > 1:
                print(f"Frequency: {peak_freq} Hz, Amplitude: {amplitude}")

            if high_pitch_min <= peak_freq <= high_pitch_max:
                print(f"High-pitched sine wave detected! Frequency: {peak_freq} Hz, Amplitude: {amplitude}")

            print(f"Frequency: {peak_freq} Hz, Amplitude: {amplitude}")

            # Sleep for 0.25 seconds
            time.sleep(0.25)

            

            # Optionally, plot the signal and frequency spectrum
            # plt.subplot(2, 1, 1)
            # plt.plot(data)
            # plt.title("Signal")
            # plt.subplot(2, 1, 2)
            # plt.plot(freqs, magnitude)
            # plt.title("Frequency Spectrum")
            # plt.show()

    except KeyboardInterrupt:
        # Stop streaming on keyboard interrupt
        stream.stop_stream()
        stream.close()
        p.terminate()

    return


In [ ]:
device_index = 0 # Replace with the index of the Aggregate Device or Multi-Output Device
device_info = p.get_device_info_by_index(device_index)
sample_rate = int(device_info['defaultSampleRate'])
number_channels = int(device_info['maxInputChannels'])
# stream = p.open(format=pyaudio.paInt16, channels=num_channels, rate=sample_rate, input=True, frames_per_buffer=1024, input_device_index=device_index)


In [21]:
# Create a slider for frequency control
freq_slider = widgets.FloatSlider(
    value=gqrx.get_frequency(),
    min=2.3e9,  # 2.3 GHz
    max=2.5e9,  # 2.5 GHz
    step=1e6,  # 1 MHz step
    description='Frequency:',
    continuous_update=False
)
widgets.interactive(update_frequency, freq=freq_slider)

stream_audio(num_channels=number_channels, sample_rate=sample_rate, device_index=device_index)


Frequency: 515.625 Hz, Amplitude: 15.0625
Frequency: 515.625 Hz, Amplitude: 15.0625
Frequency: 515.625 Hz, Amplitude: 24.21875
Frequency: 515.625 Hz, Amplitude: 24.21875
Frequency: 515.625 Hz, Amplitude: 41.3369140625
Frequency: 515.625 Hz, Amplitude: 41.3369140625
Frequency: 515.625 Hz, Amplitude: 59.8984375
Frequency: 515.625 Hz, Amplitude: 59.8984375
Frequency: 515.625 Hz, Amplitude: 102.763671875
Frequency: 515.625 Hz, Amplitude: 102.763671875
Frequency: 515.625 Hz, Amplitude: 131.712890625
Frequency: 515.625 Hz, Amplitude: 131.712890625
Frequency: 796.875 Hz, Amplitude: 169.0107421875
Frequency: 796.875 Hz, Amplitude: 169.0107421875
Frequency: 796.875 Hz, Amplitude: 222.1318359375
Frequency: 796.875 Hz, Amplitude: 222.1318359375
Frequency: 796.875 Hz, Amplitude: 307.560546875
Frequency: 796.875 Hz, Amplitude: 307.560546875
Frequency: 796.875 Hz, Amplitude: 342.560546875
Frequency: 796.875 Hz, Amplitude: 342.560546875
Frequency: 796.875 Hz, Amplitude: 317.201171875
Frequency: 796.8

In [ ]:


# Create a slider for frequency control
# freq_slider = widgets.FloatSlider(
#     value=gqrx.get_frequency(),
#     min=2.3e9,  # 2.3 GHz
#     max=2.5e9,  # 2.5 GHz
#     step=1e6,  # 1 MHz step
#     description='Frequency:',
#     continuous_update=False
# )
# widgets.interactive(update_frequency, freq=freq_slider)


In [ ]:

# Close the connection
# gqrx.close()
